
# What is Machine Learning

Building a model from example inputs to make __data-driven predictions__ vs. following strictly __static program instructions__. This means system learns how to solve a problem from example data rather than programmer telling how to do it.

Before jumping in and trying to solve a problem, ask yourself this - Will employing machine learning techniques help me reach a better solution for this problem? Trying to solve a problem like *Minimum spanning tree* using ML is an overkill since to train a model you need data and time to learn and there are other efficient ways to solve this problem (Prim's algo, Kruskal's algo).

On the other hand, to solve the problem of labelling an email as spam we have to use ML algorithms as they will give better accuracy and save time rather than a person going through the data, finding best features, correlations in the data and devising his/her own way to reach a conclusion.


## What does it mean to learn?

> Any Machine learning algorithm should be able to generalise. 

For eg: 
Alice has taken a course on ML and is expected to have "learned" all about this topic. How will her teacher know that she has done so. A common way to gauge this is to give her a test/exam. She has learned if she does well in the exam. But what makes a reasonable exam?

If her teacher gives her questions on History or Quantum mechanics, her performance in this exam will __not__ be representative of her learning. On the other hand, if her teacher asks her questions that were taught in class, answering those questions will only require recalling past experiences. She must be tested on _new but related_ problems to know how well she can generalise.

## Some learning problems

<img src="ML - Types of Problems - Google.png">

src - https://developers.google.com/machine-learning/problem-framing/cases

Solving any machine learning problem has to go through the following __Machine Learning Workflow__

<img src="ML Workflow - Google.png">

src - https://developers.google.com/machine-learning/problem-framing/big-questions

Key Takeaways

- Early steps are most important
- Expect to go backwards
- Data will require some pre-processing to suit our needs
- Do not pursue a bad solution

## Decision Tree Algorithms 

- ID3: For binary features
- C4.5: Enhanced version of of ID3 to handle continuous data
- CART: Classification and Regression Trees


In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import random

%matplotlib inline

src:- https://www.kaggle.com/mig555/mushroom-classification

to determine the value of 'class'

In [4]:
mush_data = pd.read_csv('mushrooms.csv')
mush_data.describe()
mush_data[mush_data['class'] == 'p'].shape

(3916, 23)

In [5]:
mush_data.odor.unique()
label = 'class'

mush_data[mush_data['odor'] == 'n']

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
14,e,x,f,n,f,n,f,w,b,n,...,f,w,w,p,w,o,e,k,a,g
15,e,s,f,g,f,n,f,c,n,k,...,s,w,w,p,w,o,p,n,y,u
16,e,f,f,w,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
28,e,f,f,n,f,n,f,c,n,k,...,s,w,w,p,w,o,p,k,y,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8115,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,v,l
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l


In [6]:
def train_test_split(df,ratio=0.8):
    tot = df.shape[0]
    pick_train = int(tot * ratio)
    pick_test = tot - pick_train
    random_ints = random.sample(range(tot),tot)
#     print(random_ints[:pick_train])
    train = df.iloc[random_ints[:pick_train]]
    test = df.iloc[random_ints[pick_train:]]
    test_y = test[label]
    del test[label]
    return train,test,test_y

train_data,test_x,test_y = train_test_split(mush_data)

In [8]:
# for col in mush_data.columns:
#     i=0
#     mapping={}
#     for val in mush_data[col].unique():
#         mapping[val]=i
#         i+=1
#     mush_data[col] = mush_data[col].map(mapping)
# mush_data.describe()

## ID3

In [7]:
class node:
    name = ''
    decision = None
    children = list()
    size = None
    parent_val = 0
    def __init__(self,name,s):
        self.name = name
        self.size = s
        
    def append(self,n):
        if type(n) == type(self) and (n.name == self.name and n.parent_val is not self.parent_val):
            self.children.append(n)

$ entropy(feature = \alpha) = -p\log_{2}{p} - (1-p)\log_{2}{(1-p)}$

where $ p = \frac{(label)}{(feature - \alpha)}$

In [10]:
def entropy(data):
    data = data[data!=0]
    tot = np.sum(data)
    data = data/tot
    data_log = np.log2(data)
    return -np.sum(data * data_log)

$ Info Gain(feature) = entropy(data) - entropy(feature)$

In [11]:
def info_gain(df,feature):
    if df[feature].unique().shape[0] == 1:
        return 0
    data = df.copy()
    feat_vs_label_dt = data.groupby([feature,label])[feature].count().unstack(fill_value=0)
    feat_indices = feat_vs_label_dt.index
    feat_vs_label = np.array(feat_vs_label_dt)
#     print(feat_vs_label,np.sum(feat_vs_label,axis=0))
#     diff = np.absolute(feat_vs_label[:,0] - feat_vs_label[:,1])
#     totals = np.sum(feat_vs_label,axis=1)
#     print(feature,totals)
#     label_diff = np.absolute(totals[0] - totals[1])
    entropy_tot = 0
    label_entropy = entropy(np.array(df.groupby(label)[feature].count()))
    for i in range(feat_vs_label.shape[0]):
#         print('{} * entropy({}) / {}'.format(np.sum(feat_vs_label[i,:]),feat_vs_label[i,:],np.sum(feat_vs_label)))
        entropy_tot += (np.sum(feat_vs_label[i,:]) * entropy(feat_vs_label[i,:]) / np.sum(feat_vs_label))
    return label_entropy - entropy_tot

In [12]:
np.sum(train_data.groupby([label,'odor'])['odor'].count().unstack(fill_value=0),axis=0)

odor
a     302
c     143
f    1743
l     348
m      27
n    2813
p     206
s     469
y     448
dtype: int64

In [13]:
entropy(np.array(mush_data.groupby('odor')['odor'].count()))

2.3194144457106733

In [14]:
info_gain(train_data,'odor')

0.9032468766680934

In [15]:
max_gain = (0,0)
for feat in list(train_data.columns[1:]):
    gain = info_gain(train_data,feat)
    if gain > max_gain[0]:
        max_gain = (gain,feat)
max_gain

(0.9032468766680934, 'odor')

# Iterative Dichotomizer

In [16]:
def id3(data,parent,label):
    label_counts = data.groupby(label)[label].count()
    label_entropy = entropy(np.array(label_counts))
#     print(label_entropy)
    if label_entropy > 0.1:
        max_gain = (0,0)
        for feat in list(data.columns[1:]):
            gain = info_gain(data,feat)
            if gain > max_gain[0]:
                max_gain = (gain,feat)
        best_feature = max_gain[1]
        print('split on feature:{}'.format(best_feature))
        parent.name = best_feature
        parent.children = list()
        feature_values = list(data[best_feature].unique())
        for val in feature_values:
            filtered = data[data[best_feature] == val]
            decision_node = node(parent.name,filtered.shape)
            decision_node.parent_val = val
            parent.children.append(decision_node)
            id3(filtered,decision_node,label)
    else:
        counts = data.groupby(label)[parent.name].count()
#         print(data.shape)
        parent.decision = counts.idxmax()
        parent.parent_val = parent.parent_val
        print('leaf node on feature:{}, decision:{}'.format(parent.name, parent.decision))

In [17]:
decision_tree = node('root',train_data.shape)
id3(train_data,decision_tree,label)

split on feature:odor
leaf node on feature:odor, decision:e
leaf node on feature:odor, decision:p
leaf node on feature:odor, decision:p
split on feature:spore-print-color
leaf node on feature:spore-print-color, decision:e
leaf node on feature:spore-print-color, decision:e
split on feature:habitat
leaf node on feature:habitat, decision:e
leaf node on feature:habitat, decision:e
split on feature:gill-size
leaf node on feature:gill-size, decision:p
leaf node on feature:gill-size, decision:e
split on feature:cap-color
leaf node on feature:cap-color, decision:e
leaf node on feature:cap-color, decision:e
leaf node on feature:cap-color, decision:p
leaf node on feature:cap-color, decision:p
leaf node on feature:habitat, decision:e
leaf node on feature:spore-print-color, decision:e
leaf node on feature:spore-print-color, decision:p
leaf node on feature:spore-print-color, decision:e
leaf node on feature:spore-print-color, decision:e
leaf node on feature:spore-print-color, decision:e
leaf node on

In [18]:
queue = [decision_tree]
while len(queue) != 0:
    popped = queue.pop()
    if popped.decision != None:
        print('leaf-parent:{},parent-val:{}, decision:{}, size:{}'.format(popped.name,popped.parent_val,popped.decision,popped.size[0]))
    else:
        print('node:{}, parent-val:{}, {} children, size:{}'.format(popped.name,popped.parent_val, len(popped.children),popped.size[0]))
    queue.extend(popped.children)

node:odor, parent-val:0, 9 children, size:6499
leaf-parent:odor,parent-val:c, decision:p, size:143
leaf-parent:odor,parent-val:m, decision:p, size:27
leaf-parent:odor,parent-val:p, decision:p, size:206
leaf-parent:odor,parent-val:l, decision:e, size:348
leaf-parent:odor,parent-val:y, decision:p, size:448
node:spore-print-color, parent-val:n, 8 children, size:2813
leaf-parent:spore-print-color,parent-val:o, decision:e, size:42
leaf-parent:spore-print-color,parent-val:y, decision:e, size:32
leaf-parent:spore-print-color,parent-val:h, decision:e, size:34
leaf-parent:spore-print-color,parent-val:r, decision:p, size:59
leaf-parent:spore-print-color,parent-val:b, decision:e, size:39
node:habitat, parent-val:w, 5 children, size:513
leaf-parent:habitat,parent-val:p, decision:e, size:32
node:cap-color, parent-val:l, 4 children, size:47
leaf-parent:cap-color,parent-val:y, decision:p, size:6
leaf-parent:cap-color,parent-val:w, decision:p, size:8
leaf-parent:cap-color,parent-val:n, decision:e, siz

In [19]:
def predict(data,model):
    print(model.decision)
    if model.decision != None:
        return model.decision
    print(model.name)
    val = data[model.name]
    for child in model.children:
        if child.parent_val == val:
            return predict(data,child)

In [20]:
def predict(data,model):
    print(model.decision)
    if model.decision != None:
        return model.decision
    print(model.name)
    val = data[model.name]
    for child in model.children:
        if child.parent_val == val:
            return predict(data,child)

In [21]:
mush_data[(mush_data['odor'] == 'n') & (mush_data['spore-print-color'] == 'w') & (mush_data['habitat'] == 'd') & (mush_data['gill-size'] == 'b')]

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
6967,e,b,s,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
6970,e,x,s,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7195,e,k,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7292,e,f,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7717,e,b,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7779,e,k,s,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7887,e,x,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d
7984,e,f,s,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,d


In [22]:
def bernoulli_to_binomial(df, cols):
    """
    This method converts categorical feature columns into binary valued feature
    :param df: dataframe containing the input data
    :param cols: list of columns to split
    :returns res: dataframe with columns split into binary columns
    """
    res = pd.DataFrame(df)
    local = df.copy()
    for col in cols:
        vals = set(local[col])
        if len(vals)>2:
            for val in vals:
                res[col+'_'+str(val)] = pd.Series([i == val for i in local[col]]).map({True: 1, False: 0})
            del res[col]
        elif len(vals) == 2:
            lst = list(vals)
            res[col] = local[col].map({lst[0]: 0, lst[1]: 1})
    return res

In [23]:
mush_data_bino = bernoulli_to_binomial(mush_data.copy(),mush_data.columns[1:])
train,test_x,test_y = train_test_split(mush_data_bino)

In [24]:
decision_tree_bino = node('root',train.shape)
id3(train,decision_tree_bino,label)

split on feature:odor_n
split on feature:spore-print-color_r
split on feature:stalk-surface-below-ring_y
leaf node on feature:stalk-surface-below-ring_y, decision:e
split on feature:gill-size
leaf node on feature:gill-size, decision:e
leaf node on feature:gill-size, decision:p
leaf node on feature:spore-print-color_r, decision:p
split on feature:bruises
split on feature:stalk-root_c
leaf node on feature:stalk-root_c, decision:e
split on feature:stalk-root_r
split on feature:gill-spacing
leaf node on feature:gill-spacing, decision:p
leaf node on feature:gill-spacing, decision:e
leaf node on feature:stalk-root_r, decision:e
leaf node on feature:bruises, decision:p


In [25]:
predictions = []
# print(test_x['odor'])
for ex in test_x.index:
    predictions.append(predict(test_x.loc[ex],decision_tree_bino))

None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
None
stalk-root_r
None
gill-spacing
p
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
None
stalk-root_r
None
gill-spacing
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
o

None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
None
stalk-root_r
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e

None
gill-spacing
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-be

odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
None
stalk-root_c
None
stalk-root_r
e
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
spore-

None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
None
stalk-root_r
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
None
stalk-root_c
e
None
odor_n
None
spore-print-color_r
None
stalk-surface-below-ring_y
e
None
odor_n
None
bruises
p
None
odor_n
None
bruises
p
None
odor

In [26]:
actual_test = list(test_y)
print(predictions[0],actual_test[0])
len([i for i in range(len(predictions)) if predictions[i] == actual_test[i]])

e e


1623